In [2]:
#!pip install s3cmd

     |████████████████████████████████| 143kB 14.7MB/s eta 0:00:01


In [1]:
import logging
import threading
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator, ui, base, bif
from iotfunctions.base import BaseTransformer
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.enginelog import EngineLogging
from iotfunctions import pipeline as pp
from iotfunctions.stages import DataWriterSqlAlchemy, DataWriterFile
from iotfunctions.pipeline import JobController, DataAggregator

import datetime as dt
import scipy as sp
from scipy import signal
from scipy.stats import energy_distance

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import skimage as ski
from skimage import util as skiutil # for nifty windowing
%matplotlib inline

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1599: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'


In [2]:
# @hidden_cell

credentials = {
  "tenantId": "Monitor-Demo",
  "as_api_host": "https://api-beta.connectedproducts.internetofthings.ibmcloud.com",
  "as_api_key": "a-vrvzh6-am4fwylysp",
  "as_api_token": "F+PF@HQhe_N*ecS4gb",
  "config" : {
      "objectStorageEndpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net",
      "bos_runtime_bucket" : "analytics-runtime-monitor-demo-395729cb6a06",
      "bos_logs_bucket" : "analytics-logs-monitor-demo-9c6aaaf268a1"
  },
  "objectStorage": {
      "username" : "5c9b5139a83d4f68bd2fe458a2117fac",
      "password" : "5b282a68b84d070bd8674da4771998c6cb0743a4a0288129",
      "region" : "us",
      "endpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net"
  },
  "db2": {
    "username": "bluadmin",
    "password": "MjZkZTEwN2FjMWY1",
    "databaseName": "BLUDB",
    "port": 50000,
    "httpsUrl": "https://dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000",
    "host": "dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net"
  },
  "postgresql-nada": {
      "username": "ibm_cloud_7d201f19_ffd0_475b_b058_26a76cec9905",
      "password": "04cdf453585baa96c19b5e7f65c7e2762288c3c2a6043ac059283fe38a3761f1",
      "region": "us",
      "host": "0e899846-39a1-4b58-9b60-67cb5a0aada4.bkvfvtld0lmh0umkfi70.databases.appdomain.cloud",
      "port": 32698,
      "databaseName": "ibmclouddb"
  }
}
EngineLogging.configure_console_logging(logging.DEBUG)

2020-01-08T14:47:25.411 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10


In [3]:
db_schema = None
db = Database(credentials=credentials)
print (db)

2020-01-08T14:47:35.044 DEBUG iotfunctions.db.__init__ Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2020-01-08T14:47:35.045 INFO iotfunctions.db.__init__ Connection string for SqlAlchemy => db2): db2+ibm_db://bluadmin:MjZkZTEwN2FjMWY1@dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000/BLUDB;
2020-01-08T14:47:35.045 DEBUG iotfunctions.db.__init__ created a CosClient object
2020-01-08T14:47:37.365 DEBUG iotfunctions.db.__init__ Db connection established
2020-01-08T14:47:37.369 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): api-beta.connectedproducts.internetofthings.ibmcloud.com


/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2020-01-08T14:47:49.452 DEBUG urllib3.connectionpool._make_request https://api-beta.connectedproducts.internetofthings.ibmcloud.com:443 "GET /api/meta/v1/Monitor-Demo/entityType HTTP/1.1" 200 None
2020-01-08T14:47:49.989 DEBUG iotfunctions.db.http_request http request successful. status 200


In [9]:
# read it back for comparison

table = db.get_table("markus_testdata")
start_ts = dt.datetime.utcnow() - dt.timedelta(days=60)
end_ts = dt.datetime.utcnow()
df_in = db.read_table(table, None, None, None, "evt_timestamp", start_ts, end_ts)
print (table)
print (df_in.shape)

markus_testdata
(4270, 10)


In [10]:
jobsettings = {}
et2 = metadata.EntityType('markus_testdata', db, 
                          Column('TestData',Float()),
                          Column('Pressure',Float()),
                          Column('Temperature',Float()),
                          **jobsettings)

#et2._end_ts_override = [(dt.datetime.utcnow() - dt.timedelta(days=55)).strftime('%Y-%m-%d %H:%M:%S')]

# Predict Pressure from Temperature and stores prediction in KMeansOut (doesn't make much sense, just to test the mechanics)
est = estimator.SimpleRegressor(features = ['Pressure'], targets = ['Temperature'], predictions = ['KMeansOut'])
est.delete_existing_models = True
et2._functions = [est]

# make sure the results of the python expression is saved to the derived metrics table
et2._data_items.append({'columnName': 'KMeansOut', 'columnType': 'NUMBER', 'kpiFunctionId': 22856, 
                         'kpiFunctionDto': {'output': {'name': 'KMeansOut'}},
                        'name': 'KMeansOut', 'parentDataItemName': None, 'sourceTableName': 'dm_markus_testdata',
                        'tags': {},'transient': False,'type': 'DERIVED_METRIC'})

print (et2._end_ts_override)

2019-11-28T14:09:18.243 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-11-28T14:09:18.245 DEBUG iotfunctions.util.__init__ Starting trace
2019-11-28T14:09:18.248 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_markus_testdata_20191128140918
2019-11-28T14:09:18.249 DEBUG iotfunctions.util.__init__ auto_save None
2019-11-28T14:09:18.257 WARNING iotfunctions.metadata.__init__ No _db_schema specified in **kwargs. Usingdefault database schema.
2019-11-28T14:09:18.263 DEBUG iotfunctions.util.categorize_args categorizing arguments
2019-11-28T14:09:18.264 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:markus_testdata
Functions:
Granularities:
No schedules metadata
None


In [17]:
jobsettings = { #'writer_name' : pp.DataWriterSqlAlchemy, 'db': db,
               '_production_mode': False,
               '_start_ts_override' : dt.datetime.utcnow() - dt.timedelta(days=60),
               '_end_ts_override' : (dt.datetime.utcnow() - dt.timedelta(days=55)), #.strftime('%Y-%m-%d %H:%M:%S'),
               '_db_schema': 'public', 'save_trace_to_file' : True}
job = pp.JobController(et2, **jobsettings)
job.execute()

2019-11-28T11:49:55.196 DEBUG iotfunctions.pipeline.set_payload_params Setting param _production_mode on payload to False
2019-11-28T11:49:55.200 DEBUG iotfunctions.pipeline.set_payload_params Setting param _start_ts_override on payload to 2019-09-29 11:49:55.196081
2019-11-28T11:49:55.201 DEBUG iotfunctions.pipeline.set_payload_params Setting param _end_ts_override on payload to 2019-10-04 11:49:55.196089
2019-11-28T11:49:55.202 DEBUG iotfunctions.pipeline.set_payload_params Setting param _db_schema on payload to public
2019-11-28T11:49:55.202 DEBUG iotfunctions.pipeline.set_payload_params Setting param save_trace_to_file on payload to True
2019-11-28T11:49:55.203 DEBUG iotfunctions.pipeline.set_payload_params Setting param tenant_id on payload to AnalyticsServiceDev
2019-11-28T11:49:55.567 DEBUG iotfunctions.stages.get_output_list The payload has candidate data items ['deviceid', 'evt_timestamp', 'Pressure', 'TestData', 'Temperature']. The DataReader has no projection list
2019-11-28

2019-11-28T11:49:59.308 DEBUG iotfunctions.pipeline.write Executing stage SimpleRegressor.
2019-11-28T11:49:59.313 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.softlayer.net
2019-11-28T11:49:59.751 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "DELETE /analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature HTTP/1.1" 204 0
2019-11-28T11:49:59.754 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.softlayer.net


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T11:50:00.379 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "GET /analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature HTTP/1.1" 404 361
2019-11-28T11:50:00.381 WARNING iotfunctions.util._cos_api_request error cos_api_request: request_url=https://s3-api.us-geo.objectstorage.softlayer.net/analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature, http_method=GET, status_code=404, response_text=<?xml version="1.0" encoding="UTF-8" standalone="yes"?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Resource>/analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature</Resource><RequestId>ffb78066-e307-48a3-89e0-a09153d725ad</RequestId><httpStatusCode>404</httpStatusCode></Error>
2019-11-28T11:50:00.399 ERROR iotfunctions.db.cos_load Not able to GET model.markus_testda

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

2019-11-28T11:50:25.565 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature HTTP/1.1" 200 0
2019-11-28T11:50:25.567 DEBUG iotfunctions.base.evaluate_and_write_model  wrote new model model.markus_testdata.SimpleRegressor.Temperature 
2019-11-28T11:50:25.569 INFO iotfunctions.base.execute Finished training model model.markus_testdata.SimpleRegressor.Temperature
2019-11-28T11:50:25.580 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.softlayer.net


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T11:50:26.353 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "GET /analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature HTTP/1.1" 200 2078547
2019-11-28T11:50:27.097 INFO iotfunctions.metadata.predict predicted using model model.markus_testdata.SimpleRegressor.Temperature
2019-11-28T11:50:27.100 DEBUG iotfunctions.util.log_df_info After adding predictions for target Temperature df count: 4900  ; index: id,evt_timestamp  ; columns: Temperature,Pressure,deviceid,_timestamp,KMeansOut
2019-11-28T11:50:27.103 DEBUG iotfunctions.system_function.execute Input dataframe has columns ['Temperature', 'Pressure', 'deviceid', '_timestamp'] and index ['id', 'evt_timestamp']
2019-11-28T11:50:27.104 DEBUG iotfunctions.system_function.merge_dataframe Merging dataframe with columns ['Temperature', 'Pressure', 'deviceid', '_timestamp'] and index ['id', 'evt_timestamp']
2019-11-28T11:50:27.109 DEBUG

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T11:50:28.065 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-analyticsservicedev-799d2008b460/AnalyticsServiceDev/markus_testdata/20191128/markus_testdata_trace_114955 HTTP/1.1" 200 0
2019-11-28T11:50:28.080 DEBUG iotfunctions.util.save Saved trace to cos AnalyticsServiceDev/markus_testdata/20191128/markus_testdata_trace_114955
2019-11-28T11:50:28.082 DEBUG iotfunctions.util.save wrote trace to file auto_trace_markus_testdata_20191128114955.json
2019-11-28T11:50:28.438 DEBUG iotfunctions.pipeline.update Updated job log (markus_testdata,5min): 2019-11-28 11:49:55.583217
2019-11-28T11:50:28.439 DEBUG iotfunctions.pipeline.get_next_future_execution Next scheduled execution date is 2019-11-28 11:54:55.583217
2019-11-28T11:50:28.441 DEBUG iotfunctions.pipeline.execute Ending job normally as there are no scheduled executions  due before execution end time
2019-11-28T11:50:28.502 DEBUG iotfunctions.util.run_au

In [11]:
print (et2._end_ts_override)

None


In [13]:
# @hidden_cell
!s3cmd --access_key=58ddd86b5de8468b819d385046f17033 --secret_key=ee0d6c5521ce9ff100f91b0e37d4eb8cc1a038b5a6d05b38 --host=https://s3-api.us-geo.objectstorage.softlayer.net --host-bucket=s3-api.us-geo.objectstorage.softlayer.net ls s3://analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature

2019-11-28 11:50   2078547   s3://analytics-runtime-analyticsservicedev-799d2008b460/model.markus_testdata.SimpleRegressor.Temperature


In [44]:
#entitySettings = {'_db': db, 'logical_name' : 'Client04'}
meta = db.get_entity_type('Clients04')

2019-11-28T17:34:09.297 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-11-28T17:34:09.304 DEBUG iotfunctions.util.__init__ Starting trace
2019-11-28T17:34:09.305 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_Clients04_20191128173409
2019-11-28T17:34:09.306 DEBUG iotfunctions.util.__init__ auto_save None
2019-11-28T17:34:09.307 DEBUG iotfunctions.util.categorize_args categorizing arguments


/opt/conda/envs/Python36/lib/python3.6/site-packages/sqlalchemy/engine/reflection.py:919: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'clients04'
  "columns for table '%s'" % (flavor, c, table_name)


2019-11-28T17:34:10.961 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:clients04
Functions:
Granularities:
No schedules metadata


In [56]:
outputName = 'predict'
est = estimator.SimpleRegressor(features = ['pressure'], targets = ['temperature'], predictions = [outputName])
est.delete_existing_models = True
meta._functions = [est]

# make sure the results of the python expression is saved to the derived metrics table
meta._data_items.append({'columnName': outputName, 'columnType': 'NUMBER', 'kpiFunctionId': 22856, 
                         'kpiFunctionDto': {'output': {'name': outputName}},
                        'name': outputName, 'parentDataItemName': None, 'sourceTableName': 'dm_markus_testdata',
                        'tags': {},'transient': True,'type': 'DERIVED_METRIC'})

print (meta._end_ts_override)

2019-11-27 17:44:05.093035


In [57]:
meta._functions

In [59]:
jobsettings = { #'writer_name' : pp.DataWriterSqlAlchemy, 'db': db,
               '_production_mode': False,
               '_start_ts_override' : dt.datetime.utcnow() - dt.timedelta(days=10),
               '_end_ts_override' : (dt.datetime.utcnow() - dt.timedelta(days=1)), #.strftime('%Y-%m-%d %H:%M:%S'),
                '_db_schema': 'BLUADMIN',
               'save_trace_to_file' : True}
job = pp.JobController(meta, **jobsettings)
job.execute()

2019-11-28T17:46:05.577 DEBUG iotfunctions.pipeline.set_payload_params Setting param _production_mode on payload to False
2019-11-28T17:46:05.579 DEBUG iotfunctions.pipeline.set_payload_params Setting param _start_ts_override on payload to 2019-11-18 17:46:05.577572
2019-11-28T17:46:05.581 DEBUG iotfunctions.pipeline.set_payload_params Setting param _end_ts_override on payload to 2019-11-27 17:46:05.577580
2019-11-28T17:46:05.582 DEBUG iotfunctions.pipeline.set_payload_params Setting param _db_schema on payload to BLUADMIN
2019-11-28T17:46:05.583 DEBUG iotfunctions.pipeline.set_payload_params Setting param save_trace_to_file on payload to True
2019-11-28T17:46:05.583 DEBUG iotfunctions.pipeline.set_payload_params Setting param tenant_id on payload to Monitor-Demo
2019-11-28T17:46:05.925 DEBUG iotfunctions.stages.get_output_list The payload has candidate data items ['deviceid', 'evt_timestamp', 'b_press_y', 'turbine_id', 'press_y', 'a_press_y', 'temp_x', 'temperature', 'step', 'b_temp_x

2019-11-28T17:46:08.722 DEBUG iotfunctions.system_function.execute columns considered when dropping null rows ['temperature', 'pressure']
2019-11-28T17:46:08.723 DEBUG iotfunctions.system_function.execute temperature count not null: 2840
2019-11-28T17:46:08.727 DEBUG iotfunctions.system_function.execute pressure count not null: 2840
2019-11-28T17:46:08.734 DEBUG iotfunctions.pipeline.exec_stage_method No method get_column_map on drop_null returning default None. 'DropNull' object has no attribute 'get_column_map'
2019-11-28T17:46:08.739 INFO iotfunctions.pipeline.update_last_entry Trace message:  Completed stage.
2019-11-28T17:46:08.741 INFO iotfunctions.pipeline.update_last_entry Trace payload: {'produces_output_items': False, 'output_items': None, 'discard_prior_data': False, 'new_data_items_info': 'Function is configured not to produce any new data items  during execution', 'can_proceed': True, 'updated': datetime.datetime(2019, 11, 28, 17, 46, 8, 739706), 'cumulative_usage': 11360}

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T17:46:09.311 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "DELETE /analytics-runtime-monitor-demo-395729cb6a06/model.clients04.SimpleRegressor.temperature HTTP/1.1" 204 0
2019-11-28T17:46:09.321 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.softlayer.net


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

2019-11-28T17:46:41.804 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-monitor-demo-395729cb6a06/model.clients04.SimpleRegressor.temperature HTTP/1.1" 200 0
2019-11-28T17:46:41.806 DEBUG iotfunctions.base.evaluate_and_write_model  wrote new model model.clients04.SimpleRegressor.temperature 
2019-11-28T17:46:41.822 INFO iotfunctions.base.execute Finished training model model.clients04.SimpleRegressor.temperature
2019-11-28T17:46:41.826 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.softlayer.net


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T17:46:42.429 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "GET /analytics-runtime-monitor-demo-395729cb6a06/model.clients04.SimpleRegressor.temperature HTTP/1.1" 200 1031640
2019-11-28T17:46:45.009 INFO iotfunctions.metadata.predict predicted using model model.clients04.SimpleRegressor.temperature
2019-11-28T17:46:45.012 DEBUG iotfunctions.util.log_df_info After adding predictions for target temperature df count: 1828  ; index: id,evt_timestamp  ; columns: temperature,pressure,deviceid,_timestamp,predict
2019-11-28T17:46:45.014 DEBUG iotfunctions.system_function.execute Input dataframe has columns ['temperature', 'pressure', 'deviceid', '_timestamp'] and index ['id', 'evt_timestamp']
2019-11-28T17:46:45.015 DEBUG iotfunctions.system_function.merge_dataframe Merging dataframe with columns ['temperature', 'pressure', 'deviceid', '_timestamp'] and index ['id', 'evt_timestamp']
2019-11-28T17:46:45.017 DEBUG iotfunctions.system_

/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-11-28T17:46:45.932 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-monitor-demo-395729cb6a06/Monitor-Demo/Clients04/20191128/Clients04_trace_174605 HTTP/1.1" 200 0
2019-11-28T17:46:45.941 DEBUG iotfunctions.util.save Saved trace to cos Monitor-Demo/Clients04/20191128/Clients04_trace_174605
2019-11-28T17:46:45.942 DEBUG iotfunctions.util.save wrote trace to file auto_trace_Clients04_20191128174605.json
2019-11-28T17:46:46.404 DEBUG iotfunctions.pipeline.update Updated job log (Clients04,5min): 2019-11-28 17:46:05.944054
2019-11-28T17:46:46.406 DEBUG iotfunctions.pipeline.get_next_future_execution Next scheduled execution date is 2019-11-28 17:51:05.944054
2019-11-28T17:46:46.409 DEBUG iotfunctions.pipeline.execute Ending job normally as there are no scheduled executions  due before execution end time
2019-11-28T17:46:46.504 DEBUG iotfunctions.util.run_auto_save auto_trace_Clients04_20191128174605 autosave thre

In [60]:
# @hidden_cell
!s3cmd --access_key=5c9b5139a83d4f68bd2fe458a2117fac --secret_key=5b282a68b84d070bd8674da4771998c6cb0743a4a0288129 --host=https://s3-api.us-geo.objectstorage.softlayer.net --host-bucket=s3-api.us-geo.objectstorage.softlayer.net ls s3:///analytics-runtime-monitor-demo-395729cb6a06/model.clients04.SimpleRegressor.temperature

2019-11-28 17:46   1031640   s3://analytics-runtime-monitor-demo-395729cb6a06/model.clients04.SimpleRegressor.temperature
